One interesting thing to know about the work plan species is their connection to State Species of Greatest Conservation Need. The USGS builds and maintains a synthesis of state species, linking species names to taxonomic authorities (ITIS and WoRMS) to produce a synthesized National list for each decadal reporting period (with some periodic updates during the intervening years). This notebook leverages an sgcn module in the bispy package to search the National List SGCN API. It returns and caches the summarized National List records, which include the list of states that have the species in their conservation planning process.

In [1]:
import requests
import bispy
from IPython.display import display
from joblib import Parallel, delayed
from collections import Counter

sgcn = bispy.sgcn.Search()
bis_utils = bispy.bis.Utils()

import helperfunctions

In [2]:
name_list = helperfunctions.workplan_species()

In [3]:
# Use joblib to run multiple requests for SGCN records in parallel via scientific names
sgcn_results = Parallel(n_jobs=8)(delayed(sgcn.search)(name, name_source) for name, name_source in name_list)


In [4]:
sgcn_results_filtered = [r for r in sgcn_results if r["Processing Metadata"]["Status"] == "success"]

In [5]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("cache/sgcn.json", sgcn_results_filtered))

{'Doc Cache File': 'cache/sgcn.json',
 'Document Number 112': {'Processing Metadata': {'Date Processed': '2019-07-10T14:31:07.192421',
   'Name Source': 'Lookup Name',
   'Scientific Name': 'Procambarus barbiger',
   'Search URL': 'https://sciencebase.usgs.gov/staging/bis/api/v1/swap/nationallist?scientificname=Procambarus barbiger',
   'Status': 'success',
   'Summary Result': 'Name Match'},
  'SGCN Species': {'acceptedauthorityurl': 'https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value=97578',
   'commonname': 'Jackson prairie crayfish',
   'gid': 4308,
   'matchmethod': 'Exact Match',
   'scientificname': 'Procambarus barbiger',
   'sgcn2005': 1,
   'sgcn2015': 1,
   'statelist_2005': 'Mississippi',
   'statelist_2015': 'Mississippi',
   'taxonomicgroup': 'Crustaceans',
   'taxonomicrank': 'Species'}},
 'Number of Documents in Cache': 308}

# Interesting Facet
As a point of reference, we were able to find a number of additional matches to the SGCN species based on having established a linkage to a valid ITIS record. The following code block provides a quick reference on the number of SGCN matches found (high number linked to state species of conservation need as we would expect) and the number found by having the ITIS information included in the name lookup.

In [6]:
Counter(spp["Processing Metadata"]["Name Source"] for spp in sgcn_results_filtered)

Counter({'Lookup Name': 293, 'Valid ITIS Scientific Name': 15})